In [1]:
#| hide
import sys
sys.path.append('../__pypackages__/3.9/lib/')
print(sys.path)

['/afs/crc.nd.edu/group/TAI/Users/painswor/nbdev-framework-example/nbs', '/opt/anaconda3/lib/python39.zip', '/opt/anaconda3/lib/python3.9', '/opt/anaconda3/lib/python3.9/lib-dynload', '', '/afs/crc.nd.edu/user/p/painswor/.local/lib/python3.9/site-packages', '/opt/anaconda3/lib/python3.9/site-packages', '/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/afs/crc.nd.edu/user/p/painswor/.ipython', '../__pypackages__/3.9/lib/']


# Cleaning Data

## Importing Data

In [2]:
import pandas as pd

First lets define where our data is located

In [17]:
data_location = "../data/cleaned-data"

And define where to store our cleaned data

In [18]:
cleaned_data = "../data/cleaned-data"

Now we can import our data

In [7]:
raw_data = pd.read_csv(f'{data_location}/Maintenance_Text_data.csv')
raw_data.shape

(2763, 34)

## Feature Extraction

Select the columns of relevance

In [8]:
df = pd.DataFrame()
df['text'] = raw_data['c119']
df['label'] = raw_data['c78']
df

,text,label
0,TAILWHEEL COCKED RIGHT PRIOR TO TKOF. ...,AU
1,TOW PLANE BECAME AIRBORNE THEN SETTLED.STUDENT...,ME
2,"2ND ILS APCH,ACFT'S G/S INOP.LOM TUNED TO WRON...",AU
3,PLT NOTED SOFT R BRAKE PEDAL DRG TAXI TO TKOF....,AU
4,TAXI OFF HARD SFC DUE TFC R MAIN GR BROKE THRO...,AF
...,...,...
2758,(-23) A/C RELOCATED TO NEW HANGAR TO CHECK SIZ...,II
2759,(-23) ON 2/23/08 @ APPROXIMATELY 2130 DURING T...,AF
2760,(-23) PILOT TOOK OFF FOR LEESBURG AIRPORT AND ...,II
2761,(-23) OWNER FORGOT TO FASTEN THE LOWER LEFT 4 ...,II


## Cleaning Dataframe

In [9]:
df.isna().sum()

text     15
label     0
dtype: int64

Remove NaN values

In [10]:
df = df.fillna('Null')
df = df[df['text'] != 'Null']

Check there are no missing values left

In [11]:
df.isna().sum()

text     0
label    0
dtype: int64

In [12]:
df.describe(include=[object])

,text,label
count,2748,2748
unique,2742,8
top,FORCED LANDING AFTER POWER LOSS. FOUND WATER I...,II
freq,4,1942


Remove rows with one occurance

In [13]:
counts = df['label'].value_counts()
df = df[df['label'].isin(counts[counts > 1].index)]

## Splitting Data

In [22]:
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

We'll split data into training (60%), testing (20%), and validating (20%)

In [23]:
X, y = df['text'], df['label']

In [24]:
for i, (train_index, test_index) in enumerate(ss.split(X, y)):
    X_train , X_test = X.iloc[train_index],X.iloc[test_index]
    y_train , y_test = y.iloc[train_index] , y.iloc[test_index]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.20, random_state=0)
    
    Encoder = LabelEncoder()
    y_train = Encoder.fit_transform(y_train)
    y_test = Encoder.fit_transform(y_test)
    y_val_encode = Encoder.fit_transform(y_val)
    
    final_train = pd.DataFrame({'text':X_train,'label':y_train})
    final_test = pd.DataFrame({'text':X_test,'label':y_test})
    final_val = pd.DataFrame({'text':X_val,'label':y_val_encode})
    
    final_train.to_csv(f'{cleaned_data}/train/FAA-{i}.csv', index=False)
    final_test.to_csv(f'{cleaned_data}/test/FAA-{i}.csv', index=False)
    final_val.to_csv(f'{cleaned_data}/val/FAA-{i}.csv', index=False)
    y_val.to_csv(f'{cleaned_data}/actual/FAA-{i}.csv', index=False)